<a href="https://colab.research.google.com/github/stefanoiervese/DL_Project/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.getcwd()
path='/content/drive/MyDrive/mathematics_dataset-v1.0/train-easy/arithmetic__add_or_sub.txt'
#os.listdir(path)
with open(path, "r") as file:
    # Leggi il contenuto del file
    content = file.read()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import random

data_list = [x for x in content.split('\n')]
data_list=data_list[:-1]
len_data=len(data_list)
quest=[]
ans=[]
for i in range(len_data):
  if(i%2==0):
    quest.append(data_list[i])
  else:
    ans.append(data_list[i])
coppie = list(zip(quest,ans))
random.shuffle(coppie)
quest, ans=zip(*coppie)
l=int(len(quest)/3)
train_q=quest[:2*l]
test_q=quest[2*l:]
train_a=ans[:2*l]
test_a=ans[2*l:]

voc_size=len([sentence.split() for sentence in data_list]) #1333332





In [ ]:
import torch
from gensim.models import Word2Vec

tokens = [sentence.split() for sentence in data_list]
model = Word2Vec(sentences=tokens, vector_size=2, window=2, min_count=1, sg=0)

#word_embedding = model.wv["Add"]
#print("Embedding:", word_embedding)
#parola_simile, similarita = model.wv.most_similar([word_embedding], topn=1)[0]
#print("Parola simile:", parola_simile)

qt=[]
at=[]
for x in train_q:
  qt.append(torch.tensor(model.wv[x.split()]))
for x in train_a:
  at.append(torch.tensor(model.wv[x.split()]))

# come trovare parole simili
#similar_words = model.wv.most_similar("the")
#print("Parole simili:", similar_words)

In [ ]:

import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pack_sequence, pad_sequence


class Dataset(Dataset):
    def __init__(self, questions, answers):
        self.questions = questions
        self.answers = answers

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        return self.questions[idx], self.answers[idx]

batch_size = 3
train_dataset = Dataset(qt,at)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,collate_fn=lambda batch: (
    pad_sequence([item[0] for item in batch], batch_first=True),
    pad_sequence([item[1] for item in batch], batch_first=True)
))
test_dataset = Dataset(test_q,test_a)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)


In [ ]:
net=nn.Linear(3,3)
a=torch.tensor([    [[2.,3.,4.],[1.,2.,3.]]    ,   [[2.,4.,3.],[1.,2.,3.]]   ,  [[2.,4.,3.],[1.,2.,3.]] ,[[2.,3.,4.],[1.,2.,3.]]  ])
a.shape

net(a)

b=torch.tensor([1.,2.])
net1=nn.Dropout(0.9)
net1(b)

tensor([0., 0.])

In [ ]:

import torch
import torch.nn as nn
import math
import pytorch_lightning as pl

class MultiHeadAttention(pl.LightningModule):
    def __init__(self, emb_dim, num_heads,dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        assert emb_dim % num_heads == 0, "emb_dim must be divisible by num_heads"

        self.emb_dim = emb_dim
        self.num_heads = num_heads
        self.head_dim = self.emb_dim // num_heads

        # Inizializzazione dei moduli lineari per proiettare Q, K, V e l'output.
        self.W_q = nn.Linear(self.emb_dim, self.emb_dim)
        self.W_k = nn.Linear(self.emb_dim, self.emb_dim)
        self.W_v = nn.Linear(self.emb_dim, self.emb_dim)
        self.W_o = nn.Linear(self.emb_dim, self.emb_dim)


    def forward(self, q,k,v, mask=None):
        q = self.split_heads(self.W_q(q))
        k = self.split_heads(self.W_k(k))
        v = self.split_heads(self.W_v(v))

        att = self.att_score(q, k, v, mask)
        out = self.W_o(self.combine_heads(att))

        return out

    def att_score(self, q, k, v, mask=None):
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, v)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, head_dim = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.emb_dim)







class EncoderLayer(pl.LightningModule):
    def __init__(self, emb_dim, num_heads, feedforward_dim=32, dropout=0.1):
        super(EncoderLayer, self).__init__()

        self.multihead_attention = MultiHeadAttention(emb_dim, num_heads)

        self.feedforward = nn.Sequential(
            nn.Linear(emb_dim, feedforward_dim),
            nn.ReLU(),
            nn.Linear(feedforward_dim, emb_dim)
        )

        self.layer_norm1 = nn.LayerNorm(emb_dim)
        self.layer_norm2 = nn.LayerNorm(emb_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):

        att = self.multihead_attention(x,x,x, mask)

        add_nor = self.layer_norm1(x + self.dropout(att))
        ff_out = self.feedforward(add_nor)
        out = self.layer_norm2(add_nor + self.dropout(ff_out))


        return out





class Encoder(pl.LightningModule):
    def __init__(self, emb_dim, num_heads, num_layers=6, feedforward_dim=32, dropout=0.1):
        super(Encoder, self).__init__()

        self.layers = nn.ModuleList([
            EncoderLayer(emb_dim, num_heads, feedforward_dim, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer(x, mask)
        return x


class DecoderLayer(pl.LightningModule):
    def __init__(self, emb_dim, num_heads, feedforward_dim=32, dropout=0.1):
        super(DecoderLayer, self).__init__()

        # Self-Attention Layer (Auto-Attention)
        self.self_attention = MultiHeadAttention(emb_dim, num_heads)

        # Cross-Attention Layer (Attenzione incrociata con l'encoder)
        self.cross_attention = MultiHeadAttention(emb_dim, num_heads)

        # Feedforward Neural Network Layer
        self.feedforward = nn.Sequential(
            nn.Linear(emb_dim, feedforward_dim),
            nn.ReLU(),
            nn.Linear(feedforward_dim, emb_dim)
        )

        # Layer Normalization
        self.layer_norm1 = nn.LayerNorm(emb_dim)
        self.layer_norm2 = nn.LayerNorm(emb_dim)
        self.layer_norm3 = nn.LayerNorm(emb_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):

        attn_output = self.self_attention(x, x, x, tgt_mask)
        out1 = self.layer_norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attention(out1, encoder_output, encoder_output, src_mask)
        out2 = self.layer_norm2(out1 + self.dropout(attn_output))
        ff_output = self.feedforward(x)
        out3 = self.layer_norm3(out2 + self.dropout(ff_output))
        return out3

class Decoder(pl.LightningModule):
    def __init__(self, num_layers, emb_dim, num_heads, feedforward_dim=32, dropout=0.1):
        super(Decoder, self).__init__()


        self.layers = nn.ModuleList([
            DecoderLayer(emb_dim, num_heads, feedforward_dim, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return x


class Transformer(pl.LightningModule):
    def __init__(self,tgt_vocab_size, emb_dim, num_heads, num_layers=6, feedforward_dim=32, dropout=0.1):
        super(Transformer, self).__init__()

        self.num_heads=num_heads
        self.num_layers=num_layers
        self.emb_dim=emb_dim
        self.feedforward_dim=feedforward_dim
        self.dropout=dropout

        #self.src_embedding = nn.Embedding(src_vocab_size, emb_dim)
        #self.tgt_embedding = nn.Embedding(tgt_vocab_size, emb_dim)

        self.encoder = Encoder(emb_dim, self.num_heads, num_layers, feedforward_dim, dropout)

        self.decoder = Decoder(emb_dim, num_heads, num_layers, feedforward_dim, dropout)
        self.output_layer = nn.Linear(emb_dim, tgt_vocab_size)

        #softmax!

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):

        #src_embedded = self.src_embedding(src)
        #tgt_embedded = self.tgt_embedding(tgt)

        # Codifica dell'input tramite l'encoder
        encoder_output = self.encoder(src, src_mask)

        # Decodifica tramite il decoder con attenzione incrociata
        decoder_output = self.decoder(tgt, encoder_output, src_mask, tgt_mask)

        output = self.output_layer(decoder_output)

        return output

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = torch.nn.functional.mse_loss(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def train_dataloader(self):
        return DataLoader(train_dataset, batch_size=batch_size, shuffle=True,collate_fn=lambda batch: (
    pad_sequence([item[0] for item in batch], batch_first=True),
    pad_sequence([item[1] for item in batch], batch_first=True)
))

    def val_dataloader(self):
        return DataLoader(test_dataset,batch_size=batch_size,shuffle=True)



In [ ]:

#model=Transformer(voc_size,voc_size,4,2)
#data = next(iter(train_loader))
#questions, answers = data

#model(questions,answers)



In [ ]:
import torch.optim as optim

transformer = Transformer(voc_size,4,2)

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(questions,answers)
    loss = criterion(output.contiguous().view(-1, voc_size), answers.contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

AssertionError: ignored

In [ ]:
t=Transformer(voc_size,voc_size,4,2)
